<a href="https://colab.research.google.com/github/jiieunshin/lecture_deeplearning/blob/master/bert_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 이는 구글 코랩으로 돌린 버전입니다. 그리고 기본 코드는 SKT Brain의 KoBERT를 그대로 가져왔고, 학습 및 테스트 데이터셋만 따로 준비한 것입니다.
# SKT Brain github 주소는 다음과 같습니다. https://github.com/SKTBrain/KoBERT
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3 # 최신 버전으로 설치하면 "Input: must be Tensor, not str" 라는 에러 발생
!pip install torch
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

     |████▉                           | 7.1MB 5.2MB/s eta 0:00:08
ERROR: Operation cancelled by user
     |████████████████████████████████| 348kB 6.6MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595700 sha256=7e8fbd3290efe1f297fc21ba6d491dd12e86a7afb8c70e65460629512d6617f4
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.2MB 5.1MB/s 
     |████████████████████████████████| 757kB 6.7MB/s 
     |████████████████████████████████| 901kB 9.6MB/s 
     |████████████████████████████████| 3.0MB 22.8MB/s 
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
  

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

ModuleNotFoundError: ignored

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")
bertmodel, vocab = get_pytorch_kobert_model()
from google.colab import drive
drive.mount('/content/drive')
# 학습용 데이터셋 불러오기
import pandas as pd
train_data = pd.read_csv("/content/drive/MyDrive/bigdata_deeplearning/data/train_data.csv")
test_data = pd.read_csv("/content/drive/MyDrive/bigdata_deeplearning/data/test_data.csv")

In [ ]:
train_data['카테고리'].value_counts()

In [ ]:
train_data

In [ ]:
train_data.shape

In [ ]:
# 라벨링된 카테고리명 매핑 ex) {0: '경제민주화', 1: '교통/건축/국토', 2: '기타',3: '농산어촌', 4: '문화/예술/체육/언론'}
# mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
# mapping

In [ ]:
# Train / Test set 분리
# from sklearn.model_selection import train_test_split
# train, test = train_test_split(new_data, test_size=0.2, random_state=42)
# train, test = train_test_split(new_data, test_size=0.2, random_state=42)
# print("train shape is:", len(train))
# print("test shape is:", len(test))

In [ ]:
# train.to_csv('/content/drive/MyDrive/bigdata_deeplearning/data/train_data.csv', header = True, index = False)
# test.to_csv('/content/drive/MyDrive/bigdata_deeplearning/data/test_data.csv', header = True, index = False)

In [ ]:
# 기본 Bert tokenizer 사용
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 
        self.sentences = [transform(i) for i in dataset['내용']]
        self.labels = [np.int32(i) for i in dataset['카테고리']]
    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
    def __len__(self):
        return (len(self.labels))
        

In [ ]:
# Setting parameters
max_len = 30 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 124
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5


In [ ]:
bert_train = BERTDataset(train_data, tok, max_len, True, False)
bert_test = BERTDataset(test_data, tok, max_len, True, False)

In [ ]:
print(bert_train)
print(bert_test)

In [ ]:
# pytorch용 DataLoader 사용
train_dataloader = torch.utils.data.DataLoader(bert_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(bert_test, batch_size=batch_size, num_workers=5)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 28,
                 num_classes = 5, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()
    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()
    return train_acc
# /max_indices.size()[0]

In [ ]:
# 모델 학습 시작
for e in range(num_epochs):
    train_acc = 0.0
    # test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (7847)))

In [ ]:
# 테스트 문장 예측
test_sentence = test["내용"][0]
test_label = test["카테고리"][0] # 실제 질병

In [ ]:
test["내용"]

In [ ]:
# pd.DataFrame([[test['제목'][0], test['카테고리'][0]]])

In [ ]:
# test

In [ ]:
# unseen_test = pd.DataFrame([[test_sentence, test_label]], columns = [['제목', '카테고리']])
# unseen_values = unseen_test.values
# test_set = BERTDataset(unseen_values, 0, 1, tok, max_len, True, False)
# test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=5)

In [ ]:
test_input = torch.utils.data.DataLoader(data_test, batch_size=1, num_workers=5)

In [ ]:
# BERTDataset(unseen_values, tok, max_len, True, False)

In [ ]:
# tqdm_notebook(test_input)

In [ ]:
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length = valid_length
  out = model(token_ids, valid_length, segment_ids)
  print(out)
  
  model.eval() # 평가 모드로 변경
  
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    oken_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length
    label = label.long().to(device)
    token_ids = token_ids.to(device)
    out = model(token_ids, valid_length, segment_ids)
    test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {} batch {}".format(e+1, test_acc, batch_id+1))

In [ ]:
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length = valid_length
  out = model(token_ids, valid_length, segment_ids)
  
  if batch_id == 3731:
    print(out)


In [ ]:
# 모델 학습 시작
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
  optimizer.zero_grad()
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  label = label.long().to(device)
  out = model(token_ids, valid_length, segment_ids)
  loss = loss_fn(out, label)
  loss.backward()
  torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
  optimizer.step()
  scheduler.step()  # Update learning rate schedule
  test_acc += calc_accuracy(out, label)
  if batch_id % log_interval == 0:
    print("batch id {} loss {} test acc {}".format(batch_id+1, loss.data.cpu().numpy(), test_acc / (batch_id+1)))